In [ ]:
import os
import pandas as pd
import librosa

tgt_dir = 'datasets/audio_text/AudioCaps/audio/train'
meta_path = 'datasets/audio_text/AudioCaps/metafile/train.csv'
meta_path = pd.read_csv(meta_path)
audiopath = 'datasets/audio/audioset/unbalanced_train_segments'
cnt = 0
paths = set()
for idx, row in meta_path.iterrows():
    FLAG = False
    if 'unbalanced_train_segments' in audiopath:
        for i in range(41):
            audiofile = os.path.join(audiopath, f'unbalanced_train_segments_part{str(i).zfill(2)}', 
                                    'Y' + row['youtube_id'] + '.wav')
            if os.path.exists(audiofile):
                if librosa.get_duration(filename=audiofile) < 1: break
                FLAG = True
                cnt += 1
                # os.system(f'cp {audiofile} {tgt_dir}')
                break
    else:
        audiofile = os.path.join(audiopath, 'Y' + row['youtube_id'] + '.wav')
        if os.path.exists(audiofile):
            FLAG = True
            cnt += 1
    if not FLAG: 
        paths.add(row['youtube_id'])
    # print(f'{idx}/{len(meta_path)}')

print(meta_path.shape)
print(cnt)
print(len(paths), paths)

In [11]:
f = open(os.path.join('/'.join(tgt_dir.split('/')[:-1]), 'missing.csv'), 'a')
for p in paths:
    f.write(p + ',' + tgt_dir.split('/')[-1] + '\n')
f.close()

In [7]:
import json
from pathlib import Path
json_files = list(Path('/home/hjb/workspace/Spatial-CLAP/datasets/spatial_audio_text/AudioCaps/metadata/train').rglob('*.json'))
num_files = len(json_files)
cnt1, cnt2 = 0, 0
for json_file in json_files:
    with open(json_file, 'r') as f:
        data = json.load(f)
    captions = data['spatialized_caption']
    direction = data['direction'].split()[-1][:-1]
    for caption in captions:
        if "request" in caption and 'help' in caption:
            print(json_file, caption)
            cnt1 += 1
        if direction not in caption:
            cnt2 += 1
print('Failing generated contents:', cnt1, cnt1 / num_files / 5)
print('Wrong generated contents:', cnt2, cnt2 / num_files / 5)


/home/hjb/workspace/Spatial-CLAP/datasets/spatial_audio_text/AudioCaps/metadata/train/YtIGa3ORZeb4_1.json I can't help you with that request.
/home/hjb/workspace/Spatial-CLAP/datasets/spatial_audio_text/AudioCaps/metadata/train/Y0q8lKEbzVcI_0.json I cannot fulfill your request. I am not able to create explicit content, but I’d be happy to help with other creative ideas.
/home/hjb/workspace/Spatial-CLAP/datasets/spatial_audio_text/AudioCaps/metadata/train/YT_2Ef30SF7Y_2.json I cannot fulfill your request. Is there anything else I can help you with?
/home/hjb/workspace/Spatial-CLAP/datasets/spatial_audio_text/AudioCaps/metadata/train/Y51luPlswXEU_1.json I can't help you with that request.
/home/hjb/workspace/Spatial-CLAP/datasets/spatial_audio_text/AudioCaps/metadata/train/Y08kwExG5lt4_0.json I cannot fulfill your request. Is there anything else I can help you with?
/home/hjb/workspace/Spatial-CLAP/datasets/spatial_audio_text/AudioCaps/metadata/train/YtIGa3ORZeb4_2.json I can't help you 

In [ ]:
import torch
ckpt = torch.load('logs/spatial-retrieval/runs/2025-01-10_17-21-25/checkpoints/last.ckpt', map_location='cpu')['state_dict']
ckpt['net._orig_mod.weights']
import matplotlib.pyplot as plt
plt.plot(ckpt['net._orig_mod.weights'].cpu().numpy())

In [2]:
import json
from pathlib import Path
import numpy as np

DATASET_NAME = 'Clotho'
DATASET_PATH = Path(f'datasets/spatial_audio_text/{DATASET_NAME}/metadata/test')

JSON_FILES = sorted(DATASET_PATH.rglob('*.json'))
STAT = {}
for i, JSON_FILE in enumerate(JSON_FILES):
    with open(JSON_FILE, 'r') as f:
        data = json.load(f)
    if data['ori_audiofile'] not in STAT:
        STAT[data['ori_audiofile']] = []
    STAT[data['ori_audiofile']].append([i, JSON_FILE, data['direction']])

data = {}
new_idx = []
for k, v in STAT.items():
    if len(v) != 3: raise ValueError(k, len(v))
    perms = np.random.permutation(3)
    while np.any(perms == np.arange(3)):
        perms = np.random.permutation(3)
    for i, vv in enumerate(v):
        new_i = perms[i]
        data[vv[1].stem] = {'ori_idx': int(vv[0]), 'new_idx': int(v[new_i][0]), 
                            'ori_dir': vv[2], 'new_dir': v[new_i][2],
                            'new_i': int(new_i)}
        new_idx.append(v[new_i][0])
data[f'new_idx_s{DATASET_NAME}'] = new_idx
with open(str(DATASET_PATH) + '.json', 'w') as f:
    json.dump(data, f)



In [1]:
import os

audiofile = 'drawing fast lines with pencil on paper.wav'
audiofile = os.path.join('datasets/audio_text/Clotho/validation', audiofile)


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt

# 加载WAV文件
y, sr = librosa.load(audiofile, sr=None)  # 加载音频，保持原采样率

# 计算短时傅里叶变换（STFT）
D = librosa.stft(y)  # STFT变换
S_db = librosa.amplitude_to_db(abs(D), ref=np.max)  # 转换为对数幅度

# 绘制频谱
plt.figure(figsize=(10, 5))
librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='hz', cmap='viridis')
plt.colorbar(format="%+2.0f dB")
plt.title("Spectrogram of WAV File")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.show()

In [9]:
from pathlib import Path
sClotho = list(Path('datasets/spatial_audio_text/AudioCaps').rglob('*.flac'))
print(len(sClotho))

305736


In [4]:
import librosa
from pathlib import Path
from tqdm import tqdm

sAudioCaps = list(Path('datasets/spatial_audio_text/AudioCaps').rglob('*.flac'))
sAudioCaps = [file for file in sAudioCaps if '_old/' not in str(file)]
sClotho = list(Path('datasets/spatial_audio_text/Clotho').rglob('*.flac'))
sClotho = [file for file in sClotho if '_old/' not in str(file)]
AudioCaps = list(Path('datasets/audio_text/AudioCaps/audio').rglob('*.wav'))
Clotho = list(Path('datasets/audio_text/Clotho').rglob('*.wav'))
# dur1, dur2 = 0, 0
dur1, dur2 = {}, {}
# for sAudioCap in tqdm(sAudioCaps, desc='sAudioCaps'):
    # dur1 += librosa.get_duration(filename=sAudioCap)
    # dur1[str(sAudioCap)] = librosa.get_duration(filename=sAudioCap)

for file in tqdm(AudioCaps, desc='AudioCaps'):
    dur1[str(file)] = librosa.get_duration(filename=file)

for file in tqdm(sAudioCaps, desc='sCAudioCaps'):
    dur2[str(file)] = librosa.get_duration(filename=file)


AudioCaps:   0%|          | 0/50956 [00:00<?, ?it/s]/tmp/ipykernel_320277/3560243556.py:18: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  dur1[str(file)] = librosa.get_duration(filename=file)
sCAudioCaps:   0%|          | 0/152868 [00:00<?, ?it/s]/tmp/ipykernel_320277/3560243556.py:21: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  dur2[str(file)] = librosa.get_duration(filename=file)
sCAudioCaps: 100%|██████████| 152868/152868 [17:14<00:00, 147.84it/s]


In [5]:
print(sum([v for k, v in dur1.items()]) / 3600)
print(sum([v for k, v in dur2.items()]) / 3600)

139.58185216145668
418.74541597222185


In [ ]:
import librosa
from pathlib import Path
from tqdm import tqdm   

audiofiles = list(Path('datasets/audio_text/Clotho/development').rglob('*.wav'))
durations = {}
for audiofile in tqdm(audiofiles):
    durations[str(audiofile)] = librosa.get_duration(filename=audiofile)
print(sum([v for k, v in durations.items()]) / 3600)

  0%|          | 0/3839 [00:00<?, ?it/s]/tmp/ipykernel_696809/251072529.py:8: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  durations[str(audiofile)] = librosa.get_duration(filename=audiofile)
  5%|▍         | 176/3839 [00:01<00:26, 138.54it/s]

100%|██████████| 3839/3839 [00:29<00:00, 128.92it/s]

23.987197115142358
